![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [2]:
cc_apps.shape
cc_apps.info()
cc_apps[1].unique()
print((cc_apps == '?').sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB
0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13     0
dtype: int64


In [3]:
cc_apps_imputed = cc_apps.copy()

In [4]:
cc_apps_imputed = cc_apps_imputed.replace('?', np.nan)

In [5]:
cc_apps_imputed[1] = pd.to_numeric(cc_apps_imputed[1])

In [6]:
cc_apps_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-null    float64
 2   2       690 non-null    float64
 3   3       684 non-null    object 
 4   4       684 non-null    object 
 5   5       681 non-null    object 
 6   6       681 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(3), int64(2), object(9)
memory usage: 75.6+ KB


In [7]:
for col in cc_apps_imputed:
    if cc_apps_imputed[col].dtypes == 'object':
        most_frequent = cc_apps_imputed[col].value_counts().index[0]
        cc_apps_imputed[col] = cc_apps_imputed[col].fillna(most_frequent)
    else:
        cc_apps_imputed[col] = cc_apps_imputed[col].fillna(cc_apps_imputed[col].mean())

print(cc_apps_imputed.isnull().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64


In [8]:
cc_apps_final = pd.get_dummies(cc_apps_imputed, drop_first=True)

cc_apps_final.head()

,1,2,7,10,12,0_b,3_u,3_y,4_gg,4_p,...,6_j,6_n,6_o,6_v,6_z,8_t,9_t,11_p,11_s,13_-
0,30.83,0.000,1.25,1,0,True,True,False,False,False,...,False,False,False,True,False,True,True,False,False,False
1,58.67,4.460,3.04,6,560,False,True,False,False,False,...,False,False,False,False,False,True,True,False,False,False
2,24.50,0.500,1.50,0,824,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,27.83,1.540,3.75,5,3,True,True,False,False,False,...,False,False,False,True,False,True,True,False,False,False
4,20.17,5.625,1.71,0,0,True,True,False,False,False,...,False,False,False,True,False,True,False,False,True,False


In [9]:
y = cc_apps_final.iloc[:, -1].values
x =cc_apps_final.iloc[:, :-1].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [11]:
scaler = StandardScaler()

rescaledX_train = scaler.fit_transform(X_train)

rescaledX_test = scaler.transform(X_test)

In [12]:
model = LogisticRegression()
model.fit(rescaledX_train , y_train)

y_pred = model.predict(rescaledX_test)

print(y_pred[:15])

[ True  True  True  True  True  True  True  True  True  True  True False
  True  True  True]


In [13]:
print(confusion_matrix(y_test, y_pred))

print("Accuracy Score:", model.score(rescaledX_test, y_test))

[[ 89  14]
 [ 25 100]]
Accuracy Score: 0.8289473684210527


In [14]:
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]
C = [0.01, 0.1, 1, 10]
solvers = ['liblinear', 'lbfgs']

param_grid = dict(tol=tol, max_iter=max_iter, C=C, solver=solvers)

In [15]:
grid_model = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_model_result = grid_model.fit(rescaledX_train, y_train)

In [16]:
best_score = grid_model_result.best_score_
best_params = grid_model_result.best_params_

print("Best Score: %f using %s" % (best_score, best_params))

Best Score: 0.861501 using {'C': 0.1, 'max_iter': 100, 'solver': 'liblinear', 'tol': 0.01}


In [17]:
best_model = grid_model_result.best_estimator_

In [18]:
best_score = best_model.score(rescaledX_test, y_test)

print("Final accuracy of the best model: ", best_score)

Final accuracy of the best model:  0.8333333333333334
